# Exercise #6 : Other systems

<p>You can try different systems, here are some examples.</p>

We will use the ASE Python package to build the input structures. 

<h2>6.1 : Benzene molecule</h2>

In [ ]:
# We download the pseudopotentials 
!wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.2.upf
!wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/H_ONCV_PBE-1.2.upf

In [ ]:
# We build the input for DFT
from ase import Atom, Atoms
from ase.cell import Cell
from ase.io import write

cell = Cell([[10,0,0],[0,10,0],[0,0,10]]) #Angstroms

positions = []
positions.append(Atom('C',[1.2115,0.6995,0.0000])) #Angstroms
positions.append(Atom('C',[1.2115,-0.6995,0.0000]))
positions.append(Atom('C',[0.0000,-1.3990,0.0000]))
positions.append(Atom('C',[-1.2115,-0.6995,0.0000]))
positions.append(Atom('C',[-1.2115,0.6995,0.0000]))
positions.append(Atom('H',[0.0000,2.5000,0.0000]))
positions.append(Atom('H',[2.1651,1.2500,0.0000]))
positions.append(Atom('H',[2.1651,-1.2500,0.0000]))
positions.append(Atom('H',[0.0000,-2.5000,0.0000]))
positions.append(Atom('H',[-2.1651,-1.2500,0.0000]))
positions.append(Atom('H',[-2.1651,1.2500,0.0000]))

atoms = Atoms(positions,cell=cell)

pseudopotentials = {}
pseudopotentials['C']='C_ONCV_PBE-1.2.upf'
pseudopotentials['H']='H_ONCV_PBE-1.2.upf'

inp_data = {}
inp_data['calculation'] = 'scf'
inp_data['restart_mode'] = 'from_scratch'
inp_data['prefix'] = 'benzene'
inp_data['outdir'] = './'
inp_data['pseudo_dir'] = './'
inp_data['verbosity'] = 'high'
inp_data['wf_collect'] = True

inp_data['ecutwfc'] = 50
inp_data['nspin'] = 1
inp_data['nbnd'] = 40
inp_data['assume_isolated'] = 'mp'

inp_data['diago_full_acc'] = True

write('benzene.pwi',atoms,pseudopotentials=pseudopotentials,input_data=inp_data)

In [ ]:
!cat benzene.pwi

In [ ]:
# We build the input for MBPT
import yaml

west={}
west['input_west']={} 
west['input_west']['qe_prefix']='benzene'
west['input_west']['west_prefix']='benzene'
west['input_west']['outdir']='./'

west['wstat_control']={} 
west['wstat_control']['wstat_calculation']='S'
west['wstat_control']['n_pdep_eigen']=50

west['wfreq_control']={} 
west['wfreq_control']['wfreq_calculation']='XWGQ'
west['wfreq_control']['n_pdep_eigen_to_use']=50
west['wfreq_control']['qp_bandrange']=[1,5]
west['wfreq_control']['n_refreq']=300
west['wfreq_control']['ecut_refreq']=2.0

with open('west.in', 'w') as file:
    yaml.dump(west, file, sort_keys=False)

In [ ]:
!cat west.in

In [ ]:
!mpirun -n 8 pw.x -i benzene.pwi > benzene.pwo
!mpirun -n 8 wstat.x -i wstat.in > wstat.out
!mpirun -n 8 wfreq.x -i wfreq.in > wfreq.out

<h2>6.2 : H<sub>2</sub>O</h2>

In [ ]:
# We download the pseudopotentials 
!wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/O_ONCV_PBE-1.2.upf
!wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/H_ONCV_PBE-1.2.upf

In [ ]:
# We build the input for DFT, starting from a CIF file
from ase.io import read, write

atoms = read('H2O.cif')

pseudopotentials = {}
pseudopotentials['O']='O_ONCV_PBE-1.2.upf'
pseudopotentials['H']='H_ONCV_PBE-1.2.upf'

inp_data = {}
inp_data['calculation'] = 'scf'
inp_data['restart_mode'] = 'from_scratch'
inp_data['prefix'] = 'h2o'
inp_data['outdir'] = './'
inp_data['pseudo_dir'] = './'
inp_data['verbosity'] = 'high'
inp_data['wf_collect'] = True

inp_data['ecutwfc'] = 60
inp_data['nspin'] = 1
inp_data['nbnd'] = 120

inp_data['diago_full_acc'] = True

write('h2o.pwi',atoms,pseudopotentials=pseudopotentials,input_data=inp_data)

In [ ]:
!cat h2o.pwi

In [ ]:
# We build the input for MBPT

import yaml

west={}
west['input_west']={} 
west['input_west']['qe_prefix']='h2o'
west['input_west']['west_prefix']='h2o'
west['input_west']['outdir']='./'

west['wstat_control']={} 
west['wstat_control']['wstat_calculation']='S'
west['wstat_control']['n_pdep_eigen']=200

west['wfreq_control']={} 
west['wfreq_control']['wfreq_calculation']='XWGQ'
west['wfreq_control']['macropol_calculation']='C' 
# Note that for a solid we must set macropol_calculation to 'C', the default 'N' is good for molecules
west['wfreq_control']['n_pdep_eigen_to_use']=200
west['wfreq_control']['qp_bandrange']=[20,30]
west['wfreq_control']['n_refreq']=300
west['wfreq_control']['ecut_refreq']=2.0

with open('west.in', 'w') as file:
    yaml.dump(west, file, sort_keys=False)

In [ ]:
!cat west.in

In [ ]:
!mpirun -n 8 pw.x -i h2o.pwi > h2o.pwo
!mpirun -n 8 wstat.x -i wstat.in > wstat.out
!mpirun -n 8 wfreq.x -i wfreq.in > wfreq.out

<h2>6.3 : SiC solid</h2>

In [ ]:
# We download the pseudopotentials 
!wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.2.upf
!wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/Si_ONCV_PBE-1.2.upf

In [ ]:
# We build the input for DFT
from ase.build import bulk

atoms = bulk('SiC', 'zincblende', a=8.56, cubic=True) # Angstrom

pseudopotentials = {}
pseudopotentials['C'] = 'C_ONCV_PBE-1.2.upf'
pseudopotentials['Si'] = 'Si_ONCV_PBE-1.2.upf'

inp_data = {}
inp_data['calculation'] = 'scf'
inp_data['restart_mode'] = 'from_scratch'
inp_data['prefix'] = 'sic'
inp_data['outdir'] = './'
inp_data['pseudo_dir'] = './'
inp_data['verbosity'] = 'high'
inp_data['wf_collect'] = True

inp_data['ecutwfc'] = 60
inp_data['nspin'] = 1
inp_data['nbnd'] = 100

inp_data['diago_full_acc'] = True

write('sic_scf.pwi',atoms,pseudopotentials=pseudopotentials,input_data=inp_data,kpts=[8,8,8])

inp_data['calculation'] = 'nscf'
inp_data['nosym'] = True
inp_data['noinv'] = True

# Note that for systems with k-points we run a nscf calculation making sure that symmetries are turned off

write('sic_nscf.pwi',atoms,pseudopotentials=pseudopotentials,input_data=inp_data,kpts=[2,2,2])

In [ ]:
!cat sic_scf.pwi

In [ ]:
!cat sic_nscf.pwi

In [ ]:
# We build the input for MBPT
import yaml

west={}
west['input_west']={} 
west['input_west']['qe_prefix']='sic'
west['input_west']['west_prefix']='sic'
west['input_west']['outdir']='./'

west['wstat_control']={} 
west['wstat_control']['wstat_calculation']='S'
west['wstat_control']['n_pdep_eigen']=200

west['wfreq_control']={} 
west['wfreq_control']['wfreq_calculation']='XWGQ'
west['wfreq_control']['macropol_calculation']='C'
west['wfreq_control']['n_pdep_eigen_to_use']=200
west['wfreq_control']['qp_bandrange']=[20,30]
west['wfreq_control']['n_refreq']=300
west['wfreq_control']['ecut_refreq']=2.0

with open('west.in', 'w') as file:
    yaml.dump(west, file, sort_keys=False)

In [ ]:
!cat west.in

In [ ]:
!mpirun -n 8 pw.x -i sic_scf.pwi > sic_scf.pwo
!mpirun -n 8 pw.x -i sic_nscf.pwi > sic_nscf.pwo
!mpirun -n 8 wstat.x -i wstat.in > wstat.out
!mpirun -n 8 wfreq.x -i wfreq.in > wfreq.out